In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

2023-05-18 12:12:33.910799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv("training_set.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [4]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,2,Certain materials being removed from libraries...,6
1,2,2,Write a persuasive essay to a newspaper reflec...,0
2,3,2,Do you think that libraries should remove cert...,2
3,4,2,"In @DATE1's world, there are many things found...",6
4,5,2,In life you have the 'offensive things'. The l...,6


In [5]:
df['essay'][0]

"Certain materials being removed from libraries such as books, music and magazines, shouldn't be removed from the libraries. It gives people a chance to understand how the real world @CAPS2.     Having certain materials such as books and music definitly should not be removed, because most books and music can show most people how bad the statement in the book @CAPS2 or how bad the lyrics are in a song, and help that person to avoid that type of thing that the book or song @CAPS2 saying to the reader or listener. People should give every type of music at least a try and not always doubt what they hear about what people say about that type of music. I always hear about people saying how bad the band @PERSON1 A.M. @CAPS2, just because in the lyrics it talks about drugs and how much cursing each song has. Really the band @CAPS2 talking about one mans life and how he turns his life from being a drug addict to having the best life someone could ever live. People always doubted him and never g

In [6]:
temp.head()

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,2,Certain materials being removed from libraries...,6,Certain materials removed libraries books mus...,1999,470,17,4.253191,15,112,95,33,32
1,2,2,Write a persuasive essay to a newspaper reflec...,0,Write persuasive essay newspaper reflecting vi...,636,167,3,3.808383,11,35,46,15,6
2,3,2,Do you think that libraries should remove cert...,2,Do think libraries remove certain materials sh...,864,222,15,3.891892,5,52,45,17,15
3,4,2,"In world, there are many things found offensiv...",6,In world many things found offensive Everyon...,2115,468,31,4.519231,3,98,122,27,30
4,5,2,In life you have the 'offensive things'. The l...,6,In life offensive things The little stuff ge...,1814,429,35,4.228438,23,107,93,29,30


In [7]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
X_train.shape

(1260, 3)

**PREPROCESSING**

In [10]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [11]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)


In [12]:
len(train_sents)

25717

In [13]:
train_sents[0]

['I',
 'think',
 'take',
 'book',
 'donated',
 'us',
 'first',
 'place',
 'Also',
 'I',
 'dont',
 'think',
 'take',
 'helps',
 'us',
 'learn',
 'people',
 'went',
 'lifetime',
 'Look',
 'try',
 'live',
 'life',
 'would',
 'another',
 'persons',
 'shoes',
 'Try',
 'see',
 'come',
 'life',
 'style',
 'Im',
 'big',
 'reader',
 'anything',
 'wants',
 'I',
 'get',
 'one',
 'book',
 'catches',
 'eye',
 'sounds',
 'good',
 'read',
 'I',
 'look',
 'good',
 'I',
 'would',
 'take',
 'read',
 'long',
 'time',
 'I',
 'want',
 'know',
 'happens',
 'next',
 'book',
 'So',
 'I',
 'keep',
 'reading',
 'reading',
 'like',
 'middle',
 'story',
 'teacher',
 'makes',
 'stop',
 'reading',
 'I',
 'would',
 'read',
 'long',
 'takes',
 'right',
 'book',
 'I',
 'like',
 'kinds',
 'books',
 'sports',
 'sad',
 'emotional',
 'books',
 'I',
 'like',
 'reading',
 'loud',
 'helps',
 'understand',
 'little',
 'bit',
 'better']

**Preparing WORD2VEC and LSTM Model**

In [14]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [15]:
#Training Word2Vec model
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

/var/folders/kn/gnm3d9hn21d8t3q08syhwsdw0000gp/T/ipykernel_5322/1462232201.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [16]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv.get_vector(i))        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

In [17]:
training_vectors.shape

(1260, 300)

In [18]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

2023-05-18 12:13:14.530801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [19]:
training_vectors.shape

(1260, 1, 300)

**TRAINING AND PREDICTION**

In [20]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
20/20 [==============================] - 5s 12ms/step - loss: 16.7243 - mae: 3.6974
Epoch 2/150
20/20 [==============================] - 0s 12ms/step - loss: 3.2450 - mae: 1.4577
Epoch 3/150
20/20 [==============================] - 0s 12ms/step - loss: 2.7544 - mae: 1.3281
Epoch 4/150
20/20 [==============================] - 0s 12ms/step - loss: 2.6522 - mae: 1.2998
Epoch 5/150
20/20 [==============================] - 0s 12ms/step - loss: 2.5741 - mae: 1.2700
Epoch 6/150
20/20 [==============================] - 0s 12ms/step - loss: 2.4507 - mae: 1.2513
Epoch 7/150
20/20 [==============================] - 0s 13ms/step - loss: 2.4660 - mae: 1.2463
Epoch 8/150
20/20 [==============================] - 0s 12ms/step - loss: 2.3965 - mae: 1.2208
Epoch 9/150
20/20 [==============================] - 0s 13ms/step - loss: 2.4277 - mae: 1.2258
Epoch 10/150
20/20 [==============================] - 0s 13ms/step - loss: 2.3663 - mae: 1.2077
Epoch 11/150
20/20 [============================

In [21]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

17/17 [==============================] - 1s 4ms/step


array([[5.],
       [6.],
       [5.],
       [4.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [4.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [4.],
       [5.],
       [6.],
       [5.],
       [5.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [4.],
       [6.],
       [5.],
       [4.],
       [5.],
       [5.],
       [5.],
       [4.],
       [6.],
       [5.],
       [6.],
       [4.],
       [6.],
       [5.],
       [5.],
       [3.],
       [4.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [4.],
       [5.],
       [5.],
       [6.],
       [4.],
       [5.],
       [4.],
       [6.],
       [5.],
       [5.],
       [6.],
       [5.],
       [4.],
       [4.],
       [5.],
       [5.],
       [5.],
       [4.],
       [5.],
       [5.],
       [6.],
       [5.],
       [4.],
       [5.],
       [5.],
       [6.],
       [5.],
       [5.],
       [5.],